# HDA - Project 3
## TASK B1: Activity detection
This task consists of a binary classification, where a gesture denotes activity and thus the model detects wheter there is a gesture label or not (labeled in column 6).

This first cell contains the parameters that can be tuned for code execution:
- subject: select the subject on which to test the model, between [1,4];
- folder: directory name where '.mat' files are stored;
- label_col: index of feature column to be selected to perform activity detection, between [0,6];
- window_size: parameter that sets the length of temporal windows on which to perform the convolution;
- stride: step length to chose the next window.

In [1]:
subject = 1
folder = "./data/full/"
label = 0     # default for task B1
window_size = 15
stride = 15
# make_binary = True

In [2]:
import utils
import deeplearning
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

Using TensorFlow backend.


Creation of training set and test set

In [3]:
[x_train, y_train, x_test, y_test, n_features, n_classes] = utils.preprocessing(subject,
                                                                                folder,
                                                                                label,
                                                                                window_size,
                                                                                stride,
                                                                                printInfo = True,
                                                                                make_binary = True)

Training samples:  157125 
Test samples:       57536 
Features:             110
Classes: 2

TRAINING SET:
Dataset of Images have shape:  (10474, 15, 110) 
Dataset of Labels have shape:    (10474, 2) 
Fraction of labels:   [0.11065496 0.88934504]

TEST SET:
Dataset of Images have shape:  (3834, 15, 110) 
Dataset of Labels have shape:    (3834, 2) 
Fraction of labels:   [0.17736046 0.82263954]


Preparation of data in a input-suitable form

In [4]:
detection_model = deeplearning.MotionDetection((window_size,n_features,1), n_classes)
detection_model.summary() # model visualization

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 15, 110, 1)        4         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 5, 108, 50)        1700      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 2, 108, 50)        0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 2, 5400)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 2, 20)             433680    
_________________________________________________________________
lstm_2 (LSTM)                (None, 20)                3280      
_________________________________________________________________
dense_1 (Dense)              (None, 512)               10752     
__________

In [5]:
detection_model.compile(optimizer = Adam(lr=0.01), 
                        loss = "categorical_crossentropy", 
                        metrics = ["accuracy"])

input_train = x_train.reshape(x_train.shape[0], window_size, n_features, 1)
input_test = x_test.reshape(x_test.shape[0], window_size, n_features, 1)

checkpointer = ModelCheckpoint(filepath='./data/weights_d.hdf5', verbose=1, save_best_only=True)

detection_model.fit(x = input_train, 
                    y = y_train, 
                    epochs = 10, 
                    batch_size = 128,
                    verbose = 1,
                    validation_data=(input_test, y_test),
                    callbacks=[checkpointer])

Train on 10474 samples, validate on 3834 samples
Epoch 1/10
10474/10474 [==============================] - 5s 498us/step - loss: 0.2044 - acc: 0.9264 - val_loss: 0.2039 - val_acc: 0.9178

Epoch 00001: val_loss improved from inf to 0.20391, saving model to ./data/weights_d.hdf5
Epoch 2/10
10474/10474 [==============================] - 2s 209us/step - loss: 0.1618 - acc: 0.9478 - val_loss: 0.2247 - val_acc: 0.9244

Epoch 00002: val_loss did not improve
Epoch 3/10
10474/10474 [==============================] - 2s 209us/step - loss: 0.1597 - acc: 0.9489 - val_loss: 0.2438 - val_acc: 0.9197

Epoch 00003: val_loss did not improve
Epoch 4/10
10474/10474 [==============================] - 2s 209us/step - loss: 0.1574 - acc: 0.9479 - val_loss: 0.2584 - val_acc: 0.9069

Epoch 00004: val_loss did not improve
Epoch 5/10
10474/10474 [==============================] - 2s 210us/step - loss: 0.1525 - acc: 0.9525 - val_loss: 0.2305 - val_acc: 0.9275

Epoch 00005: val_loss did not improve
Epoch 6/10
104

In [6]:
y_pred = detection_model.predict(input_test)
y_pred = np.argmax(y_pred, 1)

print(classification_report(y_test, to_categorical(y_pred)))

             precision    recall  f1-score   support

          0       0.89      0.72      0.79       680
          1       0.94      0.98      0.96      3154

avg / total       0.93      0.93      0.93      3834



In [7]:
detection_model_best = load_model('./data/weights_d.hdf5')

y_pred = detection_model_best.predict(input_test)
y_pred = np.argmax(y_pred, 1)

print(classification_report(y_test, to_categorical(y_pred)))

             precision    recall  f1-score   support

          0       0.88      0.73      0.80       680
          1       0.94      0.98      0.96      3154

avg / total       0.93      0.93      0.93      3834



## Task B2: gesture recognition
This task consists of a 17-class classification, where gestures are labeled in column 6.

To tune the following parameters, refer to the first cell of task B1:
- subject: select the subject on which to test the model, between [1,4];
- folder: directory name where '.mat' files are stored;
- label_col: index of feature column to be selected to perform activity detection, between [0,6];
- window_size: parameter that sets the length of temporal windows on which to perform the convolution;
- stride: step length to chose the next window.

Here we just need to preserve the different labels, thus we set 'make_binary' to False. We have then 18 different labels, keeping into account the null class, with label 0.

In [8]:
[x_train, y_train, x_test, y_test, n_classes] = utils.preprocessing(subject,
                                                                    folder,
                                                                    label,
                                                                    window_size,
                                                                    stride,
                                                                    printInfo = True,
                                                                    make_binary = False,
                                                                    null_class = False)

Training samples:  157125 
Test samples:       57536 
Features:             110

TRAINING SET:
Dataset of Images have shape:  (9314, 15, 110) 
Dataset of Labels have shape:    (9314, 4) 
Fraction of labels:   [0.46993773 0.31028559 0.19229117 0.02748551]

TEST SET:
Dataset of Images have shape:  (3154, 15, 110) 
Dataset of Labels have shape:    (3154, 4) 
Fraction of labels:   [0.41661382 0.24793912 0.28883957 0.04660748]


In [9]:
n_classes = 4 # OVERWRITE TO BE FIXED

In [10]:
classification_model = deeplearning.MotionClassification((window_size,n_features,1), n_classes)
classification_model.summary() # model visualization

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_2 (Batch (None, 15, 110, 1)        4         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 5, 110, 50)        600       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 2, 110, 50)        0         
_________________________________________________________________
reshape_2 (Reshape)          (None, 2, 5500)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 2, 300)            6961200   
_________________________________________________________________
lstm_4 (LSTM)                (None, 300)               721200    
_________________________________________________________________
dropout_1 (Dropout)          (None, 300)               0         
__________

In [11]:
classification_model.compile(optimizer = Adam(lr=0.01), 
                             loss = "categorical_crossentropy", 
                             metrics = ["accuracy"])

input_train = x_train.reshape(x_train.shape[0], window_size, n_features, 1)
input_test = x_test.reshape(x_test.shape[0], window_size, n_features, 1)

checkpointer = ModelCheckpoint(filepath='./data/weights_c.hdf5', verbose=1, save_best_only=True)

classification_model.fit(x = input_train, 
                         y = y_train, 
                         epochs = 10, 
                         batch_size = 128,
                         verbose = 1,
                         validation_data=(input_test, y_test),
                         callbacks=[checkpointer])

Train on 9314 samples, validate on 3154 samples
Epoch 1/10
9314/9314 [==============================] - 9s 914us/step - loss: 0.4329 - acc: 0.8246 - val_loss: 0.2436 - val_acc: 0.9309

Epoch 00001: val_loss improved from inf to 0.24357, saving model to ./data/weights_c.hdf5
Epoch 2/10
9314/9314 [==============================] - 5s 503us/step - loss: 0.2765 - acc: 0.8876 - val_loss: 0.1523 - val_acc: 0.9388

Epoch 00002: val_loss improved from 0.24357 to 0.15234, saving model to ./data/weights_c.hdf5
Epoch 3/10
9314/9314 [==============================] - 5s 503us/step - loss: 0.2525 - acc: 0.9012 - val_loss: 0.2264 - val_acc: 0.9363

Epoch 00003: val_loss did not improve
Epoch 4/10
9314/9314 [==============================] - 5s 503us/step - loss: 0.2729 - acc: 0.8891 - val_loss: 0.1805 - val_acc: 0.9321

Epoch 00004: val_loss did not improve
Epoch 5/10
9314/9314 [==============================] - 5s 503us/step - loss: 0.2657 - acc: 0.8955 - val_loss: 0.1748 - val_acc: 0.9363

Epoch 0

In [12]:
y_pred = classification_model.predict(input_test)
y_pred = np.argmax(y_pred, 1)

print(classification_report(y_test, to_categorical(y_pred)))

             precision    recall  f1-score   support

          0       0.94      0.89      0.92      1314
          1       0.85      0.90      0.88       782
          2       0.95      1.00      0.97       911
          3       1.00      0.84      0.91       147

avg / total       0.93      0.92      0.92      3154



In [13]:
detection_model_best = load_model('./data/weights_c.hdf5')

y_pred = detection_model_best.predict(input_test)
y_pred = np.argmax(y_pred, 1)

print(classification_report(y_test, to_categorical(y_pred)))

             precision    recall  f1-score   support

          0       0.92      0.94      0.93      1314
          1       0.90      0.86      0.88       782
          2       0.99      0.99      0.99       911
          3       0.99      1.00      1.00       147

avg / total       0.94      0.94      0.94      3154



## Classification with null class
(detection and classification are performed together)

In [3]:
[x_train, y_train, x_test, y_test, n_features, n_classes] = utils.preprocessing(subject,
                                                                                folder,
                                                                                label,
                                                                                window_size,
                                                                                stride,
                                                                                printInfo = True,
                                                                                make_binary = False,)

Training samples:  157125 
Test samples:       57536 
Features:             110
Classes: 5

TRAINING SET:
Dataset of Images have shape:  (10474, 15, 110) 
Dataset of Labels have shape:    (10474, 5) 
Fraction of labels:   [0.11075043 0.41789192 0.27592133 0.17099484 0.02444147]

TEST SET:
Dataset of Images have shape:  (3834, 15, 110) 
Dataset of Labels have shape:    (3834, 5) 
Fraction of labels:   [0.17736046 0.342723   0.20396453 0.23761085 0.03834116]


In [5]:
classification_model = deeplearning.MotionClassification((window_size,n_features,1), n_classes)
classification_model.summary() # model visualization

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_2 (Batch (None, 15, 110, 1)        4         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 5, 110, 50)        600       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 2, 110, 50)        0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 2, 5500)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 2, 300)            6961200   
_________________________________________________________________
lstm_2 (LSTM)                (None, 300)               721200    
_________________________________________________________________
dropout_1 (Dropout)          (None, 300)               0         
__________

In [6]:
classification_model.compile(optimizer = Adam(lr=0.01), 
                             loss = "categorical_crossentropy", 
                             metrics = ["accuracy"])

input_train = x_train.reshape(x_train.shape[0], window_size, n_features, 1)
input_test = x_test.reshape(x_test.shape[0], window_size, n_features, 1)

checkpointer = ModelCheckpoint(filepath='./data/weights_dc.hdf5', verbose=1, save_best_only=True)

classification_model.fit(x = input_train, 
                         y = y_train, 
                         epochs = 10, 
                         batch_size = 300,
                         verbose = 1,
                         validation_data=(input_test, y_test),
                         callbacks=[checkpointer])

Train on 10474 samples, validate on 3834 samples
Epoch 1/10
10474/10474 [==============================] - 9s 850us/step - loss: 0.7202 - acc: 0.7326 - val_loss: 0.4568 - val_acc: 0.8638

Epoch 00001: val_loss improved from inf to 0.45680, saving model to ./data/weights_dc.hdf5
Epoch 2/10
10474/10474 [==============================] - 4s 343us/step - loss: 0.3461 - acc: 0.8675 - val_loss: 0.4182 - val_acc: 0.8876

Epoch 00002: val_loss improved from 0.45680 to 0.41824, saving model to ./data/weights_dc.hdf5
Epoch 3/10
10474/10474 [==============================] - 4s 343us/step - loss: 0.2983 - acc: 0.8893 - val_loss: 0.3834 - val_acc: 0.8714

Epoch 00003: val_loss improved from 0.41824 to 0.38337, saving model to ./data/weights_dc.hdf5
Epoch 4/10
10474/10474 [==============================] - 4s 344us/step - loss: 0.2653 - acc: 0.9011 - val_loss: 0.4114 - val_acc: 0.8761

Epoch 00004: val_loss did not improve
Epoch 5/10
10474/10474 [==============================] - 4s 345us/step - lo

In [7]:
y_pred = classification_model.predict(input_test)
y_pred = np.argmax(y_pred, 1)

print(classification_report(y_test, to_categorical(y_pred)))

             precision    recall  f1-score   support

          0       0.90      0.82      0.86       680
          1       0.88      0.95      0.92      1314
          2       0.86      0.79      0.82       782
          3       0.98      0.99      0.99       911
          4       0.91      0.99      0.95       147

avg / total       0.91      0.91      0.90      3834



In [8]:
classification_model_best = load_model('./data/weights_dc.hdf5')

y_pred = classification_model_best.predict(input_test)
y_pred = np.argmax(y_pred, 1)

print(classification_report(y_test, to_categorical(y_pred)))

             precision    recall  f1-score   support

          0       0.88      0.82      0.85       680
          1       0.88      0.94      0.91      1314
          2       0.84      0.76      0.80       782
          3       0.98      1.00      0.99       911
          4       0.91      0.92      0.92       147

avg / total       0.90      0.90      0.90      3834

